# Agent


In [65]:
import logging 

from debiasing.llm.utils import LLMMessage
from debiasing.llm.agent import Debiaser

logger = logging.getLogger()
logger.setLevel(logging.INFO)

We will create an agent with the capabilities to debiasing a gender biased text. So the task is to process any text, detect which (if any) parts of the text contain a gender bias, and then proposed debiased alternatives for that text. For that purpose we will use two tools:

1. `GENDER_BIAS_MULTI_LABEL_CLASSIFIER`: a kind of NER-multilabel classifier tool that detect text within a paragraph for a set of possible gender biases.

1. `DEBIASER`: a kind of sequence-to-sequence transfer style for biased-to-debiased gender text.

So, **what is an agent?** A piece of code that use an LLM in chat completion mode, a bunch of tools, reasoning, planning, an a while loop to solve a given task.

The first thing is the capacity to detect when the LLM is using a tool.

In [66]:
debiaser = Debiaser(provider="anthropic")

input_text = (
    "La karencita es tan tierna. "
    "Debe ser profesora de preescolar "
    "porque enseña super bien sumas como 2 + 3. "
    "Además no creo que sea ingenieria "
    "porque a las mujeres no les gusta eso"
)

input_text = "Hola, cómo estas?"

msgs = [
    LLMMessage(
        role=LLMMessage.MessageRole.USER,
        content=input_text,
    )
]

debiaser.execute_task(msgs)

2025-01-06 00:39:49 - debiasing.configs - INFO - LLM Anthropic response: {"id":"msg_01WC677Qpig1sb3gUo3oZgsm","type":"message","role":"assistant","model":"claude-3-5-sonnet-20241022","content":[{"type":"text","text":"Lo siento, pero noto que estás escribiendo en español. Soy capaz de entender español, pero las herramientas que tengo disponibles están diseñadas para analizar y trabajar con textos en inglés, específicamente para detectar y corregir sesgos de género en textos en inglés.\n\nPara poder ayudarte mejor, te sugiero que:\n1. Me escribas el texto que quieres analizar en inglés, o\n2. Me especifiques claramente qué quieres hacer (en español o inglés) y te puedo guiar sobre cómo proceder.\n\n¿Te gustaría reformular tu consulta con un texto específico en inglés que quieras analizar para sesgos de género?\n\n(I notice you're writing in Spanish. While I can understand Spanish, the tools I have available are designed to analyze and work with English texts, specifically for detecting a

'UNBIASED'